In [39]:
import sys
import numpy as np
from PNM import *
# define pi for convenience
pi = np.pi

def CreateAndSavePFM(out_path):
    width = 512
    height = 512
    numComponents = 3

    img_out = np.empty(shape=(width, height, numComponents), dtype=np.float32)
    
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = 1.0

    write_pfm(out_path, img_out)

def LoadAndSavePPM(in_path, out_path):
    img_in = read_ppm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=img_in.dtype)
    height,width,_ = img_in.shape # Retrieve height and width
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] # Copy pixels

    write_ppm(out_path, img_out)

def LoadAndSavePFM(in_path, out_path):
    img_in = read_pfm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=img_in.dtype)
    height,width,_ = img_in.shape # Retrieve height and width
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] # Copy pixels

    write_pfm(out_path, img_out)

def LoadPPMAndSavePFM(in_path, out_path):
    img_in = read_ppm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=np.float32)
    height,width,_ = img_in.shape
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:]/255.0

    write_pfm(out_path, img_out)
            
def LoadPFMAndSavePPM(in_path, out_path):
    img_in = read_pfm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=np.float32)
    height,width,_ = img_in.shape
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] * 255.0

    write_ppm(out_path, img_out.astype(np.uint8))

def create_sphere_image():
    width = 511
    height = 511
    numComponents = 3

    img_out = np.empty(shape=(width, height, numComponents), dtype=np.float32)
    # the sphere has diameter 511, the center point is the center of the image, the other part outside the sphere is black
    for y in range(height):
        for x in range(width):
            if (x - 255.5) ** 2 + (y - 255.5) ** 2 <= 255.5 ** 2:
                img_out[y, x, :] = 1.0
            else:
                img_out[y, x, :] = 0.0

    return img_out


In [40]:
environment_graph = read_pfm('../Office/Office6.pfm')
sphere = create_sphere_image()
print(sphere.shape)
print(environment_graph.shape)

(511, 511, 3)
(576, 864, 3)


In [41]:
# store the normal vector of each pixel
normal = np.empty(shape=(sphere.shape[0], sphere.shape[1], 3), dtype=np.float32)


generate the normal vector map of sphere

In [42]:
for y in range(sphere.shape[0]):
    for x in range(sphere.shape[1]):
        if sphere[y, x, 0] == 1.0:
            # convert to polar coordinates
            x_ = (x - 255.5) / 255.5
            y_ = -(y - 255.5) / 255.5
            z = np.sqrt(1 - x_ ** 2 - y_ ** 2)
            normal[y, x, :] = np.array([x_, y_, z])
# save as the graph
write_pfm( 'normal.pfm', normal)

In [43]:
print(normal[15, 65, :])

[0. 0. 0.]


In [44]:
v = np.array([0, 0, 1])
# r = 2*(n dot v)n – v
r_map = np.empty(shape=(sphere.shape[0], sphere.shape[1], 3), dtype=np.float32)
for y in range(sphere.shape[0]):
    for x in range(sphere.shape[1]):
        if sphere[y, x, 0] == 1.0:
            n = normal[y, x, :]
            r = 2 * np.dot(n, v) * n - v
            r_map[y, x, :] = r/np.linalg.norm(r)
write_pfm('../r_map.pfm', r_map)

In [45]:
print(r_map[-255, -255, :])
print(r_map[255, -255, :])


[ 0.00391388 -0.00391388  0.9999847 ]
[0.00391388 0.00391388 0.9999847 ]


In [46]:
# conver the normal map and r map to the ppm image
normal_ppm = np.empty(shape=(sphere.shape[0], sphere.shape[1], 3), dtype=np.uint8)
r_map_ppm = np.empty(shape=(sphere.shape[0], sphere.shape[1], 3), dtype=np.uint8)
for x in range(sphere.shape[0]):
    for y in range(sphere.shape[1]):
        if sphere[x, y, 0] == 1.0:
            # normal_ppm[x, y, :] = (normal[x, y, :] + 1 ) * 127.5 int 
            # r_map_ppm[x, y, :] = (r_map[x, y, :] + 1 ) * 127.5 int
            normal_ppm[x, y, :] = (normal[x, y, :]/np.linalg.norm(normal)  + 1)/2 * 127.5
            r_map_ppm[x, y, :] = (r_map[x, y, :] + 1)/2 * 127.5
write_ppm('../normal.ppm', normal_ppm)
write_ppm('../r_map.ppm', r_map_ppm)


In [47]:
# Cartesian to polar coordinates conversion
def cart2pol(x, y, z):
    phi = np.arctan(z / x)
    theta = np.arccos(y)
    if phi < 0:
        phi += np.pi
    return phi, theta


def pol2cart(phi, theta):
    x = np.sin(phi) * np.cos(theta)
    y = np.cos(phi)
    z = np.sin(theta) * np.sin(phi)
    return x, y, z

In [48]:
#flip the environment graph
# environment_graph_flip = np.flip(environment_graph, axis=1)

In [49]:
# sphere = create_sphere_image()
# record_phi_theta = np.empty(shape=(sphere.shape[0], sphere.shape[1], 2), dtype=np.float32)
# print(r_map.shape)
# for y in range(r_map.shape[0]):
#     for x in range(r_map.shape[1]):
#         if sphere[y, x, 0] == 1.0:
#             x_, y_, z = r_map[y, x, :]
#             phi, theta = cart2pol(x_, y_, z)
#             record_phi_theta[y, x, :] = np.array([phi, theta])

#             theta = theta / np.pi
#             # phi = 2*np.pi - phi
#             phi = (phi) / (2 * np.pi)
#             # phi = 0.75 - phi if phi > 0.75 else 1.75 - phi
#             sphere[y, x, :] = environment_graph[min(environment_graph.shape[0]-1,int(theta * environment_graph.shape[0])), min(environment_graph.shape[1]-1,environment_graph.shape[1] - int(phi * environment_graph.shape[1])), :]
# write_pfm('../tests.pfm', sphere)


In [50]:
sphere = create_sphere_image()
tp = create_sphere_image()
for y in range(sphere.shape[0]):
    for x in range(sphere.shape[1]):
        if sphere[y, x, 0] == 1.0:
            # convert to polar coordinates
            x_, y_, z = r_map[y, x, :]
            u = (-x_ + 1) / 2
            theta = np.arccos(y_)
            phi = np.arccos(min(1.0, max(-1.0, x_ / np.sin(theta))))
            if z < 0:
                phi = 2 * np.pi - phi

            theta = theta / np.pi
            phi = phi / 2 / np.pi
            phi = 0.75 - phi if phi < 0.75 else 1.75 - phi
            sphere[y, x, :] = environment_graph[int(min(environment_graph.shape[0] - 1, theta * environment_graph.shape[0])), int(min(environment_graph.shape[1] - 1, phi * environment_graph.shape[1])), :]
            tp[y, x, :] = np.array([theta, phi, 0])

# sphere = sphere ** (1 / 2.2)
write_pfm('ghost.pfm',sphere)
# write_pfm(tp, 'tp.pfm')

In [51]:
# sphere = create_sphere_image()
# tp = create_sphere_image()
# for y in range(sphere.shape[0]):
#     for x in range(sphere.shape[1]):
#         if sphere[y, x, 0] == 1.0:# convert to polar coordinates
#             x_,y_,z = r_map[y,x,:]
#         u=(-x_+1) /2
#         theta = np.arccos(y_)
#         phi = np.arccos(min(1.0, max(-1.0,x_ / np.sin(theta))))
#         if z <0:
#             phi = 2 *np.pi - phi
#         theta = theta / np.pi
#         phi = phi / 2 / np.pi
#         phi = 0.75 - phi if phi < 0.75 else 1.75 - phi
#         # pherely,x, =eviroment_graphit mn enviroment.grept.stapel0 -1, theta  eniroment.grepi.stepel0), nt nnewiroment. grapm.stepell -1, phi * eniromment.graph.stapelil)),tply, x, :] = np.array([theta, phi, 0])
#         sphere[y, x, :] = environment_graph[int(phi * environment_graph.shape[0]), int(theta * environment_graph.shape[1]), :]
# write_pfm('../tests.pfm', sphere)

In [52]:
cart2pol(-0.1, 0, -1)

(1.4711276743037347, 1.5707963267948966)

In [53]:

# # # print the range of phi and theta
# print(np.min(record_phi_theta[:,:,0]))
# print(np.max(record_phi_theta[:,:,0]))
# print(np.min(record_phi_theta[:,:,1]))
# print(np.max(record_phi_theta[:,:,1]))

            

In [54]:
np.tan(pi/2)

1.633123935319537e+16

In [55]:
#print the range of x, y and z in the r map
print(np.min(r_map[:,:,0]))
print(np.max(r_map[:,:,0]))
print(np.min(r_map[:,:,1]))
print(np.max(r_map[:,:,1]))
print(np.min(r_map[:,:,2]))
print(np.max(r_map[:,:,2]))

-0.9999945
0.9999945
-0.9999945
0.9999945
-0.9998851
0.9999847


In [56]:
sphere[0,1,:] = np.array([1,1,1])

In [57]:
write_pfm('../sphere.pfm', sphere)